In [56]:
import tensorflow as tf
# tf.config.list_physical_devices('GPU')

In [57]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D # we want only 2 classes open eyes and close eyes and we want only learning not output layer
from tensorflow.keras.preprocessing.image import ImageDataGenerator #data augmentation


In [58]:
BATCH_SIZE =8
EPOCHS=5

In [59]:
train_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 0.2,shear_range = 0.2,
    zoom_range = 0.2,width_shift_range = 0.2,
    height_shift_range = 0.2, validation_split = 0.2) # create 5 images from each images and 20% data goes for validation

Train data 

In [60]:
train_data=train_datagen.flow_from_directory(r'C:\Users\virus\OneDrive\Desktop\Proj-Drowsiness-Detection\Prepared_Data\Train',target_size=(80,80),batch_size=BATCH_SIZE,class_mode='categorical',subset='training')

validation_data=train_datagen.flow_from_directory(r'C:\Users\virus\OneDrive\Desktop\Proj-Drowsiness-Detection\Prepared_Data\Train',target_size=(80,80),batch_size=BATCH_SIZE,class_mode='categorical',subset='validation')

Found 61519 images belonging to 2 classes.
Found 15379 images belonging to 2 classes.


#### test data 
- no need to generate each 5 images like train_data

In [61]:
test_datagen=ImageDataGenerator(rescale=1./255)
test_data =test_datagen.flow_from_directory(r'C:\Users\virus\OneDrive\Desktop\Proj-Drowsiness-Detection\Prepared_Data\Test',target_size=(80,80),batch_size=BATCH_SIZE,class_mode='categorical')

Found 8000 images belonging to 2 classes.


- we want only base model
- include_top = False bcz: we don't want last layer
- weights = 'imagenet':  means which knowlege
- input_tensor = Input(shape = (80,80,3)   : 1 for grey images 3 for color images  we gave 3 bcz lighting condition

In [62]:
bmodel =InceptionV3(include_top=False,weights='imagenet',input_tensor=Input(shape=(80,80,3)))
hmodel =bmodel.output
hmodel =Flatten()(hmodel) # convert into 1 colum array (n*1) n rows 1 column
hmodel = Dense(64,activation='relu')(hmodel) # relu for take only important data
hmodel=Dropout(0.5)(hmodel) # avoid overfitting
hmodel=Dense(2,activation='softmax')(hmodel)  # 2 for classification model 

model=Model(inputs=bmodel.input,outputs=hmodel)
for layer in bmodel.layers:
    layer.trainable=False  # we only want learn not train

In [ ]:
# bmodel.summary()

model.summary()

In [64]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [65]:
checkpoint=ModelCheckpoint(r'C:\Users\virus\OneDrive\Desktop\Proj-Drowsiness-Detection\models\best_model.keras',monitor='val_loss',save_best_only=True,verbose=3)
#verbose is just font progress
earlystop = EarlyStopping(monitor='val_loss',patience=7,verbose=3,restore_best_weights=True)

learning_rate =ReduceLROnPlateau(monitor='val_loss',patience=3,verbose=3) #how much speed it learning
callbacks = [checkpoint, earlystop, learning_rate]

In [66]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(train_data,steps_per_epoch = train_data.samples// BATCH_SIZE,
                   validation_data = validation_data,
                   validation_steps = validation_data.samples// BATCH_SIZE,
                   callbacks = callbacks,
                    epochs = EPOCHS)

Epoch 1/5
7689/7689 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.9009 - loss: 0.2445
Epoch 1: val_loss improved from inf to 0.22957, saving model to C:\Users\virus\OneDrive\Desktop\Proj-Drowsiness-Detection\models\best_model.keras
7689/7689 ━━━━━━━━━━━━━━━━━━━━ 1284s 166ms/step - accuracy: 0.9009 - loss: 0.2445 - val_accuracy: 0.9006 - val_loss: 0.2296 - learning_rate: 0.0010
Epoch 2/5
   1/7689 ━━━━━━━━━━━━━━━━━━━━ 14:08 110ms/step - accuracy: 1.0000 - loss: 0.0537

c:\Users\virus\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:155: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 2: val_loss improved from 0.22957 to 0.19994, saving model to C:\Users\virus\OneDrive\Desktop\Proj-Drowsiness-Detection\models\best_model.keras
7689/7689 ━━━━━━━━━━━━━━━━━━━━ 4s 519us/step - accuracy: 1.0000 - loss: 0.0537 - val_accuracy: 1.0000 - val_loss: 0.1999 - learning_rate: 0.0010
Epoch 3/5
7689/7689 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.9275 - loss: 0.1814
Epoch 3: val_loss did not improve from 0.19994
7689/7689 ━━━━━━━━━━━━━━━━━━━━ 920s 119ms/step - accuracy: 0.9275 - loss: 0.1814 - val_accuracy: 0.9119 - val_loss: 0.2048 - learning_rate: 0.0010
Epoch 4/5
   1/7689 ━━━━━━━━━━━━━━━━━━━━ 14:10 111ms/step - accuracy: 0.8750 - loss: 0.1106
Epoch 4: val_loss did not improve from 0.19994
7689/7689 ━━━━━━━━━━━━━━━━━━━━ 1s 98us/step - accuracy: 0.8750 - loss: 0.1106 - val_accuracy: 0.6667 - val_loss: 0.9993 - learning_rate: 0.0010
Epoch 5/5
7689/7689 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.9355 - loss: 0.1682
Epoch 5: val_loss improved from 0.19994 to 0.19045

10 to 50 epochs

# model evaluation


In [ ]:
acc_tr, loss_tr = model.evaluate(train_data)
print(acc_tr)
print(loss_tr)

In [ ]:
acc_vr, loss_vr = model.evaluate(validation_data)
print(acc_vr)
print(loss_vr)

In [ ]:
acc_test, loss_test = model.evaluate(test_data)
print(acc_tr)
print(loss_tr)